In [55]:
import matplotlib.pyplot as plt
import netCDF4
import xarray as xr
import os, glob
import numpy as np
from scipy.interpolate import griddata
import h5py
import pandas as pd
import cftime

import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

## Static vars

In [56]:
peps = 1e-4 #epsilon in mm/h for IMERG log-normalization

IMROOT = '/glade/work/jpan/IMERG_vecs/'
GOROOT = '/glade/work/jpan/GOES_vecs/'
WTROOT = './' #weights folder

CHS = ['C%s' + num for num in ['08', '09', '10', '13']]
CHS

['C%s08', 'C%s09', 'C%s10', 'C%s13']

## Open input datasets

In [60]:
#GOES_dss = [netCDF4.MFDataset(os.path.join(GOROOT, ch % '' + '*.nc'), aggdim='time') for ch in CHS]
GOES_dss = [xr.open_mfdataset(os.path.join(GOROOT, ch % '' + '*.nc'), combine='nested', concat_dim='time') for ch in CHS] #list of xr datasets
for ii, gd in enumerate(GOES_dss):
    #gd['time'] = pd.to_datetime(gd.time.values, format='%Y-%m-%d-%H:%M:%S')
    gd['time'] = [cftime.datetime.strptime(tstr, '%Y-%m-%d-%H:%M:%S', calendar='standard') for tstr in gd.time.values]
    GOES_dss[ii] = gd.sortby('time').drop_duplicates('time')
print(GOES_dss[0])

IM_ds = xr.open_mfdataset(os.path.join(IMROOT, '*.nc')) #single xr dataset
#IM_ds['time'] = pd.to_datetime(IM_ds['time'].values)
print(IM_ds)

<xarray.Dataset> Size: 10GB
Dimensions:   (time: 26141, len: 97714)
Coordinates:
  * time      (time) object 209kB 2018-01-01 00:11:20 ... 2018-09-30 23:56:18
  * len       (len) int64 782kB 0 1 2 3 4 5 ... 97709 97710 97711 97712 97713
Data variables:
    rad_vecs  (time, len) float32 10GB dask.array<chunksize=(2904, 97714), meta=np.ndarray>
Attributes:
    description:  ABI radiance vectors
<xarray.Dataset> Size: 6GB
Dimensions:  (time: 13104, idx: 39125)
Coordinates:
  * time     (time) object 105kB 2018-01-01 00:00:00 ... 2018-09-30 23:30:00
  * idx      (idx) int64 313kB 0 1 2 3 4 5 ... 39120 39121 39122 39123 39124
Data variables:
    pmmhr    (time, idx) float32 2GB dask.array<chunksize=(1488, 39125), meta=np.ndarray>
    lat      (time, idx) float32 2GB dask.array<chunksize=(1488, 39125), meta=np.ndarray>
    lon      (time, idx) float32 2GB dask.array<chunksize=(1488, 39125), meta=np.ndarray>


## Match IMERG and GOES timestamps

In [ ]:
for ii, gd in enumerate(GOES_dss):
    minutes = gd.time.dt.minute
    min_mask = ((minutes >= 25) & (minutes <= 34)) | (minutes >= 55) | (minutes <= 4)
    GOES_dss[ii] = gd.where(min_mask, drop=True)

fewest = np.argmin([gd.time.shape[0] for gd in GOES_dss]) #which dataset has the fewest timestamps?
few_times = GOES_dss[fewest].time
GOES_dss = [gd.sel(time=few_times, method='nearest') for gd in GOES_dss]
IM_ds = IM_ds.sel(time=few_times, method='nearest')

In [ ]:
[gd.time.shape for gd in GOES_dss], IM_ds.time.shape

# _________________________________________________________________________________

In [ ]:
CMROOT = r"C:\Users\axt5780\OneDrive - The Pennsylvania State University\PIML_project\IMERG_vectors"
CMROOT = '/glade/work/jpan/IMERG_vecs/'
nc_files = glob.glob(os.path.join(CMROOT, "*.nc"))

all_data = []
for nc_file in nc_files:
    channel_vec = netCDF4.Dataset(nc_file)
    data = channel_vec['pmmhr'][:, :]
    
    if np.isnan(data).any():
        valid_mask = ~np.isnan(data)
        nan_mask = np.isnan(data)  
        
        x, y = np.meshgrid(np.arange(data.shape[1]), np.arange(data.shape[0]))
      
        interpolated_data = griddata(
              points=(x[valid_mask], y[valid_mask]),
              values=data[valid_mask],
              xi=(x[nan_mask], y[nan_mask]),
              method='linear'
        )
        
        data[nan_mask] = interpolated_data
        
    all_data.append(data)
    channel_vec.close()
    
train = np.concatenate(all_data, axis=0)

print("Combined data shape:", train.shape)

In [ ]:
log_tr = np.log(train + EPS)
tr_log_norm = (log_tr - np.min(log_tr)) / (np.max(log_tr) - np.min(log_tr))

In [ ]:
tr_log_norm

In [ ]:
class Autoencoder(Model):
  '''def __init__(self, latent_dim, shape):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.shape = shape
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='sigmoid'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(tf.math.reduce_prod(shape).numpy(), activation='sigmoid'),
      layers.Reshape(shape)
    ])'''

  def __init__(self, latent_dim, shape):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.shape = shape
    self.encoder = tf.keras.Sequential([
      layers.Flatten(), # 64k
      layers.Dense(latent_dim*8, activation='swish'), # 64k -> latent_dim*8
      layers.Dense(latent_dim, activation='swish'), # latent_dim*8 -> laten_dim
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(latent_dim*8, activation='swish'), # latent_dim -> latent_dim*8
      layers.Dense(tf.math.reduce_prod(shape).numpy()), # latent_dim*8 -> vector w/o activation
      layers.Reshape(shape)
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [ ]:
shape = train.shape[1:]
latent_dim = 64

In [ ]:
autoencoder = Autoencoder(latent_dim, shape)

autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

history = autoencoder.fit(tr_log_norm, tr_log_norm,
              epochs=50,
          shuffle=True)

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.legend()

In [ ]:
autoencoder.save_weights(f'weights/IMERG_{latent_dim}_downsample_log.weights.h5')
print("Weights saved successfully.")

## REOPENING THE MODEL WITH WEIGHTS

In [ ]:
ae1 = Autoencoder(latent_dim, shape) 
ae1.predict(tr_log_norm)
ae1.load_weights(f'weights/IMERG_{latent_dim}_downsample_log.weights.h5')

In [ ]:
[ae1.weights[ii] - autoencoder.weights[ii] for ii in range(len(ae1.weights))]

#### Loaded weights of the new model match the weights of the original model

# Check whether the distribution of encoded data matches original data

In [ ]:
pred[:, ::100].ravel()

In [ ]:
ae1.summary()

In [ ]:
plt.hist(train[::1000, ::100].ravel(), bins=10**np.arange(-3, 2, 0.5))
plt.xscale('log')
plt.ylim(0, 150)
plt.ylabel('count')
plt.xlabel('p rate [mm/h]')

In [ ]:
pred_log_norm = ae1.predict(tr_log_norm[::1000])
log_pred = pred_log_norm * (np.max(log_tr) - np.min(log_tr)) + np.min(log_tr)
pred = np.exp(log_pred) - EPS

In [ ]:
plt.hist(pred[:, ::100].ravel(), bins=10**np.arange(-3, 2, 0.5))
plt.xscale('log')
plt.ylim(0, 150)
plt.ylabel('count')
plt.xlabel('p rate [mm/h]')

In [ ]:
(pred[:, ::100] < EPS).sum()

In [ ]:
(train[::1000, ::100] < EPS).sum()